## Apartado 1 - Clasificación de texto según el autor
Construid dos modelos de diferentes autores: Josep Carner y Miquel dels Sants Oliver. Para ello, podéis usar sus obras disponibles en el Proyecto Gutenberg. Por ejemplo, de Carner, tomad la traducción de los cuentos de Mark Twain. Luego, clasificad frases en el estilo de cada uno que muestren cómo vuestro modelo las identifica correctamente.

In [4]:
from my_utils import open_data
from text import *

hostal = open_data("CA-text/hostal.txt").read()
wordseq = words(hostal)

P_Miquel = UnigramWordModel(wordseq, 5)

adventures = open_data("CA-text/adventures.txt").read()
wordseq = words(adventures)

P_Josep = UnigramWordModel(wordseq, 5)

In [5]:
from learning import NaiveBayesLearner

dist = {('Miquel', 1): P_Miquel, ('Josep', 1): P_Josep}

nBS = NaiveBayesLearner(dist, simple=True)

/Users/davidramirez/Documents/IEDIB/02 Modelos IA/T5 NLP/.venv/lib/python3.10/site-packages/qpsolvers/solvers/__init__.py:752: UserWarning: no QP solver found on your system, you can install solvers from PyPI by ``pip install qpsolvers[open_source_solvers]``
  warnings.warn(


In [6]:
def recognize(sentence, nBS):
    sentence = sentence.lower()
    sentence_words = words(sentence)
    
    return nBS(sentence_words)

Primero he probado a usar una frase extraída de los archivos CA-text/hostal.txt y CA-text/adventures.txt para comprobar si el modelo es capaz de identificar correctamente el autor de la frase.

In [7]:
recognize("havia donat la direcció a un personatge destinat a fer molt de renou", nBS)

'Miquel'

In [8]:
recognize("potser mai no bastarà mentre el món sigui món", nBS)

'Josep'

Para probar con otro ejemplo, le he preguntado al ChatGPT la diferencia de estilo entre los dos autores. A continuación, se muestra la respuesta del ChatGPT:

```
Diferencias resumidas:
	•	Josep Carner → Poètic, refinat, musical i idealitzat.
	•	Miquel dels Sants Oliver → Narratiu, directe, reflexiu i analític.
```

A raíz de esto he probado a clasificar dos frases de cada autor para ver si el modelo es capaz de diferenciar el estilo de cada uno. Pero en este caso estas frases no han sido sacadas de los archivios .txt sino generadas.

In [9]:
recognize("Els carrers, encara humits, semblaven desertar de la nit", nBS)

'Miquel'

In [10]:
recognize("Amb un gest majestuós em fità un instant", nBS)

'Josep'

## Apartado 2 - Generar texto

Construye modelos n-gram con n=1, n=3, n=5 y n=7 a partir de un texto de tu elección. Puede ser una obra del Proyecto Gutenberg o una noticia de prensa, por ejemplo. Observa cómo, a medida que aumenta el número de palabras consideradas, la plausibilidad del texto generado mejora.

He optado por elegir Don Quijote de la Mancha como corpus de entranmiento para el modelo. El texto está guardado en /aima-data/ES-text/quijote.txt

In [11]:
import random
from text import NgramWordModel

def generar_texto(model, n, longitud):
    generated_text = []
    # Elegimos una semilla aleatoria del modelo
    seed = random.choice(list(model.dictionary.keys()))
    generated_text.extend(seed)
    
    for i in range(longitud - n):
        last_sequence = tuple(generated_text[-(n-1):])  # Tomamos los últimos caracteres generados
        next_possibilities = []  # Lista para guardar opciones

        for k in model.dictionary.keys():  # Revisamos todos los n-gramas aprendidos
            if k[:n-1] == last_sequence:  # Si el inicio del n-grama coincide...
                next_possibilities.append(k[n-1])  # Guardamos la siguiente letra
        
        if not next_possibilities:  # Si no hay opciones, salimos
            break

        next = random.choice(next_possibilities)  # Elegimos una opción aleatoria
        generated_text.append(next)  # La añadimos al texto generado
    
    return ' '.join(generated_text)

# Cargar texto y entrenar el modelo
quijote = open_data("ES-text/quijote.txt").read()
wordseq = words(quijote)

Una vez preparada la función hago distintas pruebas modificando la n. Y veremos que es correcta la afirmación anterior; a medida que aumentamos el número de carácteres que se tienen en cuenta el texto generado empieza a tener más sentido.

In [12]:
model_ngram = NgramWordModel(3, wordseq)

# Generar texto
new_text = generar_texto(model_ngram, 3, 50)
print(new_text)

el cansancio deste combate o ya vengan encaminadas por la empresa y fue luego escrito al duque de austria digan que esto hab a habiendo primero con groseras ceremonias rogado a don antonio y pregunt ndose los unos de contento mira hermano cuando yo pens que todos le dijeron que


In [13]:
model_ngram = NgramWordModel(5, wordseq)

# Generar texto
new_text = generar_texto(model_ngram, 5, 50)
print(new_text)

nadie porque as como maritornes le at ella y la otra se fueron muertas de risa y le dejaron asido de manera que fue imposible soltarse estaba pues como se ha dicho castillos eran a su parecer todas las ventas donde alojaba y que la hija del ventero no s


In [14]:
model_ngram = NgramWordModel(7, wordseq)

# Generar texto
new_text = generar_texto(model_ngram, 7, 50)
print(new_text)

que hab a sabido que en aquella casa viv a nos deb a de haber hecho aquel beneficio y en se al de que lo agradec amos hecimos zalemas a uso de moros inclinando la cabeza doblando el cuerpo y poniendo los brazos sobre el pecho de all a poco


Por hacer una prueba con una n mucho más grande he decido usar 30. Vemos que da un buen resultado al tener más contexto.  
Y a pesar de que algunas frases de las que genera no tienen un sentido a nivel de significado, por lo menos sí que son correctas en gran mayoría gramaticalmente.

In [19]:
model_ngram = NgramWordModel(30, wordseq)

# Generar texto
new_text = generar_texto(model_ngram, 30, 50)
print(new_text)

d as ha que pudiera yo estar bogando en ellas no son los amores como los que vuestra merced piensa dijo el galeote que los m os fueron que quise tanto a una canasta de colar atestada de ropa blanca que la abrac conmigo tan fuertemente que a no quit


## Apartado 3 - Análisis sintáctico

Aplicad las herramientas de alguna libería de Python para analizar sintácticamente la oración siguiente:  

```
Tots els éssers humans neixen lliures i iguals en dignitat i drets.
```

In [36]:
import spacy

# Cargar el modelo de lenguaje en catalán
nlp = spacy.load("ca_core_news_sm")

# Frase a analizar
sentence = "Tots els éssers humans neixen lliures i iguals en dignitat i drets"

# Analizar la frase
doc = nlp(sentence)


# Analizar la sintaxis de la frase
print("Sintaxis:")
print("Frases nominales:", [chunk.text for chunk in doc.noun_chunks])
print("Verbos:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
print("Sujetos:", [token.text for token in doc if token.dep_ == "nsubj"])
print("Adjetivos:", [token.text for token in doc if token.pos_ == "ADJ"])
print("Preposiciones:", [token.text for token in doc if token.pos_ == "ADP"])
print("Conjunciones:", [token.text for token in doc if token.pos_ == "CCONJ"])
print()


# Mostrar información de cada palabra
print("Información de cada palabra:")
for token in doc:
    print(f"Palabra: {token.text} | Lema: {token.lemma_} | Etiqueta gramatical: {token.pos_} | Dependencia sintáctica: {token.dep_} | Gobernante: {token.head.text} \n")
    


Sintaxis:
Frases nominales: ['Tots els éssers humans', 'dignitat i drets']
Verbos: ['neixar']
Sujetos: ['éssers']
Adjetivos: ['humans', 'lliures', 'iguals']
Preposiciones: ['en']
Conjunciones: ['i', 'i']

Información de cada palabra:
Palabra: Tots | Lema: tot | Etiqueta gramatical: DET | Dependencia sintáctica: det | Gobernante: els 

Palabra: els | Lema: el | Etiqueta gramatical: DET | Dependencia sintáctica: det | Gobernante: éssers 

Palabra: éssers | Lema: ésser | Etiqueta gramatical: NOUN | Dependencia sintáctica: nsubj | Gobernante: neixen 

Palabra: humans | Lema: humà | Etiqueta gramatical: ADJ | Dependencia sintáctica: amod | Gobernante: éssers 

Palabra: neixen | Lema: neixar | Etiqueta gramatical: VERB | Dependencia sintáctica: ROOT | Gobernante: neixen 

Palabra: lliures | Lema: lliure | Etiqueta gramatical: ADJ | Dependencia sintáctica: obj | Gobernante: neixen 

Palabra: i | Lema: i | Etiqueta gramatical: CCONJ | Dependencia sintáctica: cc | Gobernante: iguals 

Palabra: 

## Apartado 4 - Recuperación de información

#### 1.	Cargad el artículo de la Wikipedia sobre Europa (https://ca.wikipedia.org/wiki/Europa) en una lista de frases y recuperad la siguiente información, buscando las frases más similares:

- Quan es va gestar el concepte d'Europa?
- Quina és l'espècie humana autòctona d'Europa?
- Quan es varen formar els estats actuals d'Europa?
- Quin clima té Europa?

Para completar el ejercicio usaré "requests" para obtener el contenido de la página y "BeautifulSoup" para extraer el texto de la página.
Con "spacy" analizaré las frases y buscaré las más similares a las preguntas usando transformers.

In [38]:

import requests
from bs4 import BeautifulSoup
import spacy
from sentence_transformers import SentenceTransformer, util

url = "https://ca.wikipedia.org/wiki/Europa"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Extraer el texto del artículo
article_text = ' '.join([p.text for p in soup.find_all(["p", "div"]) if p.text])

# Dividir en frases
nlp = spacy.load("ca_core_news_sm")
doc = nlp(article_text)
sentences = [sent.text for sent in doc.sents] 

questions = [
  "Quan es va gestar el concepte d'Europa?", 
  "Quina és l'espècie humana autòctona d'Europa?", 
  "Quan es varen formar els estats actuals d'Europa?", 
  "Quin clima té Europa?"
]

# Cargar modelo de embeddings para comparar
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Calcular similitud entre preguntas y frases del artículo
question_embeddings = model.encode(questions, convert_to_tensor=True)
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

for i, question in enumerate(questions):
    similarities = util.pytorch_cos_sim(question_embeddings[i], sentence_embeddings)
    best_idx = similarities.argmax()
    print(f"\nPregunta: {question}")
    print(f"Frase más similar: {sentences[best_idx]}")




Pregunta: Quan es va gestar el concepte d'Europa?
Frase más similar: El naixement d'Europa[modifica]
Europa, com a unitat històrica, es va gestar a l'edat mitjana.

Pregunta: Quina és l'espècie humana autòctona d'Europa?
Frase más similar: Aquesta espècie es trobava ja a Europa quan va arribar l'humà de Cromanyó (Homo sapiens), espècie a què pertany tota la humanitat actual.

Pregunta: Quan es varen formar els estats actuals d'Europa?
Frase más similar: Molts dels estats de l'Europa actual es van formar després de la Primera Guerra Mundial.

Pregunta: Quin clima té Europa?
Frase más similar: [Consulta: 14 febrer 2011].

↑ «European Climate».


#### 2.	¿Todas las preguntas tienen una respuesta adecuada? ¿Hay preguntas que convendría formular de otra manera?

Vemos que la mayoría de las preguntas tienen una respuesta bastante correcta. De hecho desde la primera hasta la tercera nos responde de manera perfecta.
El problema está en la última pregunta, parece que no acaba de dar con la respuesta que toca y simplemente nos da una referencia a un link que habla del clima.

Vamos a probar de formular la pregunta de manera distitna para ver si obtenemos una mejor respuesta.

In [ ]:
questions = [
  "Com es el clima a Europa?",
]

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

question_embeddings = model.encode(questions, convert_to_tensor=True)
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

for i, question in enumerate(questions):
    similarities = util.pytorch_cos_sim(question_embeddings[i], sentence_embeddings)
    best_idx = similarities.argmax()
    print(f"\nPregunta: {question}")
    print(f"Frase más similar: {sentences[best_idx]}")


Pregunta: Com es el clima a Europa?
Frase más similar: [Consulta: 14 febrer 2011].

↑ «European Climate».


Cambiar manera de preguntar sobre el clima parece no modificar la respuesta. Para confirmarlo voy a preguntar dando la respuesta implícita en la pregunta.

In [40]:
questions = [
  "Europa té un clima mità temperat?",
]

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

question_embeddings = model.encode(questions, convert_to_tensor=True)
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

for i, question in enumerate(questions):
    similarities = util.pytorch_cos_sim(question_embeddings[i], sentence_embeddings)
    best_idx = similarities.argmax()
    print(f"\nPregunta: {question}")
    print(f"Frase más similar: {sentences[best_idx]}")


Pregunta: Europa té un clima mità temperat?
Frase más similar: [Consulta: 14 febrer 2011].

↑ «European Climate».


Pero después de investgiar cuál podría ser una manera de mejorar la respuesta me doy cuenta que la respuesta que obtenemos ahora es parte de las "Referencias" de wikipedia, al final de la web.
Esto no nos interesa ya que no contiene información relevante, por lo que se me ocurre filtrar el texto que extraemos de la web para que no contenga frases si hay carácteres entre "«»".
De esta manera evito que se devuelva información de las referencias.

Vamos a probarlo, y además usando la pregunta original.

In [41]:
sentences = [sent.text for sent in doc.sents if not any(char in sent.text for char in "«»")]

questions = [
  "Quin clima té Europa?",
]

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

question_embeddings = model.encode(questions, convert_to_tensor=True)
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

for i, question in enumerate(questions):
    similarities = util.pytorch_cos_sim(question_embeddings[i], sentence_embeddings)
    best_idx = similarities.argmax()
    print(f"\nPregunta: {question}")
    print(f"Frase más similar: {sentences[best_idx]}")


Pregunta: Quin clima té Europa?
Frase más similar: El clima d'Europa és de naturalesa temperada i continental amb un clima marítim que predomina a les costes occidentals i un clima mediterrani al sud.


Confirmamos que así obtenemos la respuesta correcta.